In [4]:
import os
import numpy as np
import skimage
from skimage import io

cwd = os.getcwd()
lbow = os.path.join(cwd, 'LLB')
lbowfiles = os.listdir(lbow)


/Users/pgb13/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [78]:
lbchannels = []
for i in range(len(lbowfiles)):
     lbchannels.append(io.imread(os.path.join(lbow, lbowfiles[i])))

In [79]:
print(lbchannels[0].dtype)
print(lbchannels[0].min())

uint16
0


In [80]:
for i in range(len(lbchannels)):
    print(lbchannels[i].shape)
    print(lbchannels[i].dtype)
    lbchannels[i] = np.amax(lbchannels[i], axis=0).astype(np.int32)
    print(lbchannels[i].shape)
    print(lbchannels[i].dtype)

(12, 1997, 1997)
uint16
(1997, 1997)
int32
(12, 1997, 1997)
uint16
(1997, 1997)
int32
(12, 1997, 1997)
uint16
(1997, 1997)
int32


In [81]:
channelscopy = np.copy(lbchannels)

for i in range(len(lbchannels)):
    print(lbchannels[i].max())
    for j in range(len(lbchannels)):
        if i != j:
            print(f'i: {i}, j: {j}')
            lbchannels[i] = np.subtract(lbchannels[i], channelscopy[j])
    print(lbchannels[i].max())
    lbchannels[i] = lbchannels[i].clip(0)

60885
i: 0, j: 1
i: 0, j: 2
58787
43633
i: 1, j: 0
i: 1, j: 2
38533
62704
i: 2, j: 0
i: 2, j: 1
60292


In [82]:
lbchannels[0]

array([[180, 290,   0, ..., 135, 223, 316],
       [390, 486, 143, ..., 305, 464, 300],
       [494, 181, 290, ..., 401, 439, 278],
       ...,
       [  0,   0, 328, ..., 375, 300, 432],
       [  6, 259, 224, ..., 206, 148,   0],
       [196, 193,   0, ..., 341,   0, 282]], dtype=int32)

In [83]:
lbchannels[0].min()

0

In [85]:
for i, channel in enumerate(lbchannels):
    io.imsave(os.path.join(lbow, f'flat_{i}.tif'), skimage.img_as_uint(channel))

/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:118: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint16.
  .format(dtypeobj_in, dtypeobj_out))
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int32 to uint16 without scaling because max value 58787 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/pgb13/Documents/Temporary Image Downloads/RRobo/LLB/flat_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int32 to uint16 without scaling because max value 38533 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:13

In [100]:
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage import img_as_uint
from skimage.color import label2rgb

In [101]:
for i, channel in enumerate(lbchannels):
    thresh = threshold_otsu(channel)
    bw = closing(channel > thresh, square(3))
    cleared = clear_border(bw)
    label_image, num_labeled = label(cleared, return_num=True)
    for region in regionprops(label_image):
        if region.area <=50:
            num_labeled -= 1
    print(num_labeled)

    io.imsave(os.path.join(lbow, "lb_channel_" + str(i) + "_bw.tif"), img_as_uint(bw))
    io.imsave(os.path.join(lbow, "lb_channel_" + str(i) + "_label_image.tif"), img_as_uint(label2rgb(label_image)))
    io.imsave(os.path.join(lbow, "lb_channel_" + str(i) + "_origional.tif"), img_as_uint(channel))

23


/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/pgb13/Documents/Temporary Image Downloads/RRobo/LLB/lb_channel_0_bw.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/pgb13/Documents/Temporary Image Downloads/RRobo/LLB/lb_channel_0_label_image.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:118: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint16.
  .format(dtypeobj_in, dtypeobj_out))
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting

9


/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/pgb13/Documents/Temporary Image Downloads/RRobo/LLB/lb_channel_1_bw.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/pgb13/Documents/Temporary Image Downloads/RRobo/LLB/lb_channel_1_label_image.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int32 to uint16 without scaling because max value 38533 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/pgb13/Documents/Temporary Image Downloads/RRobo/LLB/lb_channel_1_origional.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


596


/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int32 to uint16 without scaling because max value 60292 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/Users/pgb13/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/pgb13/Documents/Temporary Image Downloads/RRobo/LLB/lb_channel_2_origional.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
